In [0]:
import pickle
import numpy as np
import pandas as pd
from tensorflow.keras.utils import plot_model
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import LSTM, Dense, Embedding, Dropout
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.losses import SparseCategoricalCrossentropy

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
# p = pd.read_csv("drive/My Drive/pos_final.csv")
# p["tag"] = 1
# n = pd.read_csv("drive/My Drive/neg_final.csv")
# n["tag"] = 0

# dp = pd.DataFrame(p["content"].astype(str))
# dn = pd.DataFrame(n["content"].astype(str))

# x_p = dp["content"]
# x_n = dn["content"]

# y_p = p["tag"]
# y_n = n["tag"]   

# x = pd.concat([x_p, x_n])
# y = pd.concat([y_p, y_n])

# labels = list(y.unique())
# labels

In [0]:
  # 出現太少的詞，你可以選擇不看，只留出現次數最高的2000(num_words)
  # tok = Tokenizer(num_words=20000)
  # tok.fit_on_texts(x)

  # x = tok.texts_to_sequences(x)
  # x_pad = pad_sequences(x, maxlen=128)
  # y = np.array(y)

In [0]:
# x_pad.shape[1]

In [0]:
# 匯入資料
# 檔案的資料中，特徵為content, 類別為label.
def load_data():
    # df = pd.read_csv(filepath)

    p = pd.read_csv("drive/My Drive/機不可失/pos_neg/pos_chi.csv")
    p["tag"] = 1
    n = pd.read_csv("drive/My Drive/機不可失/pos_neg/neg_c.csv")
    n["tag"] = 0

    #dp = pd.DataFrame(p["content"].astype(str)) # .astype是我的檔案問題加的，若發生錯誤可刪
    #dn = pd.DataFrame(n["content"].astype(str))

    x_p = p["content"]
    x_n = n["content"]

    y_p = p["tag"]
    y_n = n["tag"]   

    x = pd.concat([x_p, x_n])
    y = pd.concat([y_p, y_n])
    
    # labels = list(y.unique())
    
    label_dictionary = {'正面': 1, '負面': 0}
    with open('drive/My Drive/label_dict.pk', 'wb') as f:
        pickle.dump(label_dictionary, f)
    # ------------------------------------------------------------------------
    # 預處理 1. 先把文字化成數字    
    # 出現太少的詞，你可以選擇不看，只留出現次數最高的20000(num_words)，可調整
    tok = Tokenizer(num_words=20000)
    tok.fit_on_texts(x)

    x = tok.texts_to_sequences(x)
    x_pad = pad_sequences(x, maxlen=128) # 評論長度自己抓
    y = np.array(y)
    # print(x_pad)
    # print(len(x_pad))
    # print(y)
    # print(len(y))

    # Saving Dictionary，此份字典，在我們訓練好並載入模組使用時，輸入端需要用同樣的字典將文字轉成數字
    # with open('drive/My Drive/tokenizer.pk', 'wb') as handle:
    # pickle.dump(tok, handle, protocol=pickle.HIGHEST_PROTOCOL)

    return x_pad, y

In [0]:
# 建立深度學習模型， Embedding + LSTM + Softmax.
def create_LSTM(n_units, input_shape, output_dim):
    x, y = load_data()
    model = Sequential()
    model.add(Embedding(input_dim=20001, output_dim=output_dim,
                        input_length=input_shape, mask_zero=True))
    model.add(LSTM(n_units)) # https://www.zhihu.com/question/41949741，LSTM參數介紹，看不懂就去睡覺吧
    model.add(Dropout(0.2))
    model.add(Dense(2, activation='softmax'))
    model.compile(loss=SparseCategoricalCrossentropy(), optimizer='adam', metrics=['accuracy'])

    plot_model(model, to_file='./model_lstm.png', show_shapes=True)
    model.summary()

    return model

In [0]:
# 模型訓練
def model_train(input_shape, filepath, model_save_path):

    # 將資料集分為訓練集和測試集，佔比為9:1
    # input_shape = 100
    x, y = load_data()
    train_x, test_x, train_y, test_y = train_test_split(x, y, test_size = 0.1, random_state = 42)

    # 模型輸入引數，需要自己根據需要調整
    n_units = 256
    batch_size = 256
    epochs = 100
    output_dim = 128

    # 模型訓練
    lstm_model = create_LSTM(n_units, input_shape, output_dim)

    callbacks = [
        EarlyStopping(patience=3, restore_best_weights=True),
    #     ModelCheckpoint("nlp.h5", save_best_only=True)
    ]

    lstm_model.fit(train_x, 
                   train_y, 
                   epochs=epochs, 
                   batch_size=batch_size, 
                   verbose=1,
                   validation_split=0.1,
                   callbacks=callbacks)

    # 模型儲存
    lstm_model.save(model_save_path)

    result = lstm_model.evaluate(test_x, test_y)
    print("測試集結果為:", result[1])
    

In [14]:
data = load_data()
data

(array([[   0,    0,    0, ...,    0,    0,  115],
        [   0,    0,    0, ...,    0,  226, 1039],
        [   0,    0,    0, ...,    0,   90, 1905],
        ...,
        [   0,    0,    0, ..., 1675,  613,   43],
        [   0,    0,    0, ...,    0,    0, 1676],
        [   0,    0,    0, ...,    0,    0, 1677]], dtype=int32),
 array([1, 1, 1, ..., 0, 0, 0]))

In [15]:
if __name__ == '__main__':
    filepath = "drive/My Drive/"
    input_shape = 128
    model_save_path = 'drive/My Drive/lstm_model.h5'
    model_train(input_shape, filepath, model_save_path)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 128, 128)          2560128   
_________________________________________________________________
lstm (LSTM)                  (None, 256)               394240    
_________________________________________________________________
dropout (Dropout)            (None, 256)               0         
_________________________________________________________________
dense (Dense)                (None, 2)                 514       
Total params: 2,954,882
Trainable params: 2,954,882
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
427/427 [==============================] - 130s 303ms/step - loss: 0.4735 - accuracy: 0.8482 - val_loss: 0.3931 - val_accuracy: 0.8635
Epoch 2/100
427/427 [==============================] - 126s 295ms/step - loss: 0.3485 - accuracy: